In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("C:\\Users\\SAHIL SHINGNE\\Desktop\\news_articles.csv")

In [3]:
df.head(2)

,author,published,title,text,language,site_url,main_img_url,type,label,title_without_stopwords,text_without_stopwords,hasImage
0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,muslims busted they stole millions in govt ben...,print they should pay all the back all the mon...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,bias,1.0,muslims busted stole millions govt benefits,print pay back money plus interest entire fami...,1.0
1,reasoning with facts,2016-10-29T08:47:11.259+03:00,re why did attorney general loretta lynch plea...,why did attorney general loretta lynch plead t...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,bias,1.0,attorney general loretta lynch plead fifth,attorney general loretta lynch plead fifth bar...,1.0


In [4]:
df.dropna(inplace=True)
df.reset_index(inplace=True)

In [5]:
x=df.drop(["label","site_url","main_img_url","index","title","text"],axis=1).copy()
y=df["label"].copy()

In [6]:
x["combine"]=x["author"]+x["language"]+x["type"]+x["title_without_stopwords"]+x["text_without_stopwords"]+str(x["hasImage"])

In [7]:
x_f=x.drop(["author","published","language","type","title_without_stopwords","text_without_stopwords","hasImage"],axis=1)

In [8]:
x_f

,combine
0,Barracuda Brigadeenglishbiasmuslims busted sto...
1,reasoning with factsenglishbiasattorney genera...
2,Barracuda Brigadeenglishbiasbreaking weiner co...
3,Fed Upenglishbiaspin drop speech father daught...
4,Fed Upenglishbiasfantastic trumps point plan r...
...,...
2039,Matt Barberenglishbiastrump vs clinton risk vs...
2040,Jane Chastainenglishbiasgingrich slutshames me...
2041,Michael Brownenglishbiasyoutube bans clintons ...
2042,Ann Coulterenglishbiaswikileaks bombshells hil...


In [9]:
corpus=list(x_f["combine"])

In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(corpus,y ,test_size=0.33,random_state=42)

In [11]:
from tensorflow.keras.preprocessing.text import one_hot

In [12]:
voc_size=5000

In [13]:
rep_train=[one_hot(word,voc_size)for word in x_train]

In [14]:
l=[]
for i in range(len(rep_train)):
    l.append(len(rep_train[i]))
maxlen=max(l)
print(maxlen)
    

3055


In [15]:
from tensorflow.keras.preprocessing.sequence import pad_sequences 

In [16]:
emd=pad_sequences(rep_train,padding="pre",maxlen=maxlen)
print(emd)

[[   0    0    0 ... 3170  318 4031]
 [   0    0    0 ... 3170  318 4031]
 [   0    0    0 ... 3170  318 4031]
 ...
 [   0    0    0 ... 3170  318 4031]
 [   0    0    0 ... 3170  318 4031]
 [   0    0    0 ... 3170  318 4031]]


In [17]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [18]:
embedding_vector_features=50
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 50)          250000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               60400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 310,501
Trainable params: 310,501
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
X_train=np.array(emd)
Y_train=np.array(y_train)

In [20]:
rep_t=[one_hot(word,voc_size)for word in x_test]

In [21]:
l1=[]
for i in range(len(rep_t)):
    l1.append(len(rep_t[i]))
maxl=max(l1)
print(maxl)

3031


In [22]:
emd1=pad_sequences(rep_t,padding="pre",maxlen=maxl)
print(emd1)

[[   0    0    0 ... 3170  318 4031]
 [   0    0    0 ... 3170  318 4031]
 [   0    0    0 ... 3170  318 4031]
 ...
 [   0    0    0 ... 3170  318 4031]
 [   0    0    0 ... 3170  318 4031]
 [   0    0    0 ... 3170  318 4031]]


In [23]:
X_test=np.array(emd1)
Y_test=np.array(y_test)

In [ ]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=10,batch_size=64)

Epoch 1/10
22/22 [==============================] - 1243s 57s/step - loss: 0.6705 - accuracy: 0.6187 - val_loss: 0.6434 - val_accuracy: 0.6578
Epoch 2/10
22/22 [==============================] - 2638s 120s/step - loss: 0.6655 - accuracy: 0.6187 - val_loss: 0.6510 - val_accuracy: 0.6578
Epoch 3/10
22/22 [==============================] - 7672s 349s/step - loss: 0.6672 - accuracy: 0.6187 - val_loss: 0.6460 - val_accuracy: 0.6578
Epoch 4/10
22/22 [==============================] - 50049s 2275s/step - loss: 0.6701 - accuracy: 0.6187 - val_loss: 0.6450 - val_accuracy: 0.6578
Epoch 5/10
11/22 [==============>...............] - ETA: 14:02 - loss: 0.6726 - accuracy: 0.6023